In [1]:
import IJulia
import Base64

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = "L2hvbWUvdml0dXJpL0RvY3VtZW50b3MvR2l0SHViL2Jsb2cvcG9zdHMtZHMvanVsaWEtZm9yLXItdXNlcnM="
  if !isempty(run_path)
    run_path = String(Base64.base64decode(run_path))
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
using RCall;

R"""
median(1:5)
"""

RObject{IntSxp}
[1] 3


In [3]:
x = [1:5;]

@rput x

R"median(x)"

RObject{IntSxp}
[1] 3


In [4]:
#| eval: false
using TidierData
using RDatasets

movies = dataset("ggplot2", "movies");

@chain movies begin
    @mutate(Budget = Budget / 1_000_000)
    @filter(Budget >= mean(skipmissing(Budget)))
    @select(Title, Budget)
    @slice(1:5)
end

In [5]:
#| eval: false

@edit maximum([1:5;])

In [6]:
#julia
# define power2 for numbers
power2(x) = x^2;

# apply in vectors
power2.(1:10)

10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [7]:
#julia
X = reshape([1:16;], (4, 4))

4×4 Matrix{Int64}:
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16

In [8]:
#julia
power2.(X)

4×4 Matrix{Int64}:
  1  25   81  169
  4  36  100  196
  9  49  121  225
 16  64  144  256

In [9]:
#julia
[1:5;] .+ 10

5-element Vector{Int64}:
 11
 12
 13
 14
 15

In [10]:
#| warning: false
#| echo: false
#R
R"""
f1_vec = function(x) {
    y = x + 1
}

f1_loop = function(x) {
    y = x
    for (i in seq_along(x)) y[i] = x[i] + 1
    y
}

f1_map = function(x) {
    purrr::map_dbl(x, \(xi) xi + 1)
}

x = 1:100000

bench::mark(
    f1_vec(x)
    ,f1_loop(x)
    ,f1_map(x)
    ,relative = TRUE
)
"""

┌ Warning: RCall.jl: Aviso: Some expressions had a GC in every iteration; so filtering is disabled.
└ @ RCall ~/.julia/packages/RCall/0ggIQ/src/io.jl:172


RObject{VecSxp}
# A tibble: 3 × 13
  expression   min median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc total_time
  <bch:expr> <dbl>  <dbl>     <dbl>     <dbl>    <dbl> <int> <dbl>   <bch:tm>
1 f1_vec(x)    1      1       122.       1        11.0   833    11      501ms
2 f1_loop(x)  42.9   13.4      11.0      1.03      1      75     1      500ms
3 f1_map(x)  529.   146.        1        1.00     34.0     7    35      515ms
# ℹ 4 more variables: result <list>, memory <list>, time <list>, gc <list>


In [11]:
#julia
f1_vec(x) = x .+ 1;

function f1_loop(x)
    y = similar(x)
    @inbounds for i ∈ eachindex(x) y[i] = x[i] + 1 end
    y
end;

function f1_map(x)
    map(x) do xi
        xi + 1 
    end
end;

In [12]:
#| eval: true
#julia
using BenchmarkTools;
x = [1:100000;];

@benchmark f1_vec($x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  51.382 μs …  2.234 ms  ┊ GC (min … max): 0.00% … 92.81%
 Time  (median):     75.726 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   85.594 μs ± 48.602 μs  ┊ GC (mean ± σ):  5.00% ±  9.14%

    ▂▂ █▂                                                      
  ▂▂██▄██▃▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂ ▃
  51.4 μs         Histogram: frequency by time         299 μs <

 Memory estimate: 781.30 KiB, allocs estimate: 2.

In [13]:
#| eval: true
#julia
@benchmark f1_loop($x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  56.761 μs … 487.885 μs  ┊ GC (min … max): 0.00% … 67.33%
 Time  (median):     74.960 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   85.545 μs ±  40.868 μs  ┊ GC (mean ± σ):  4.82% ±  9.23%

   █  ▂                                                         
  ▃██▅█▅▄▃▃▄▅▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  56.8 μs         Histogram: frequency by time          316 μs <

 Memory estimate: 781.30 KiB, allocs estimate: 2.

In [14]:
#| eval: true
#julia
@benchmark f1_map($x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  58.017 μs … 480.735 μs  ┊ GC (min … max): 0.00% … 74.99%
 Time  (median):     75.071 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   85.569 μs ±  40.296 μs  ┊ GC (mean ± σ):  4.81% ±  9.18%

   █                                                            
  ▃█▆▃▇▄▃▃▃▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂ ▂
  58 μs           Histogram: frequency by time          312 μs <

 Memory estimate: 781.30 KiB, allocs estimate: 2.

In [15]:
function f_sum(n)
    s = 0
    for i in 1:n
        s += i
    end

    s
end;

In [16]:
@time f_sum(100)

  0.000001 seconds


5050

In [17]:
@time f_sum(100_000)

  0.000001 seconds


5000050000

In [18]:
@time f_sum(100_000_000)

  0.000001 seconds


5000000050000000

In [19]:
x = 1:10

1:10

In [20]:
typeof(x)

UnitRange{Int64}

In [21]:
Base.show_supertypes(typeof(x))

UnitRange{Int64} <: AbstractUnitRange{Int64} <: OrdinalRange{Int64, Int64} <: AbstractRange{Int64} <: AbstractVector{

Int64} <: Any

In [22]:
#| eval: false
@edit maximum(1:10)

In [23]:
#| eval: false
maximum(r::AbstractUnitRange) = isempty(r) ? throw(ArgumentError("range must be non-empty")) : last(r)

In [24]:
@time sin.(1:1_000_000);

  0.018168 seconds (2 allocations: 7.629 MiB)


In [25]:
using ProgressMeter;

@showprogress for i in 1:10
    sleep(0.1)
end

Progress:  20%|████████▎                                |  ETA: 0:00:07

Progress:  90%|████████████████████████████████████▉    |  ETA: 0:00:00

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


In [26]:
#| eval: false
Threads.@threads for i = 1:10
    a[i] = Threads.threadid()
end

In [27]:
#| eval: false

using MyPackage.Reports

# or
import MyPackage.APIs as API

MD.api1()

In [28]:
[1, 2] ∩ [2, 3]

1-element Vector{Int64}:
 2

In [29]:
2 ∈ [2, 3]

true

In [30]:
f(r) = π*r^2

f(3)

28.274333882308138

In [31]:
# Euler's identity
ℯ^(im * π) + 1 |> round

0.0 + 0.0im